In [ ]:
import breeze.linalg.DenseVector
import io.github.mandar2812.dynaml.graphics.charts.Highcharts._
import io.github.mandar2812.dynaml.DynaMLPipe._
import io.github.mandar2812.dynaml.pipes._
import io.github.mandar2812.dynaml.utils.GaussianScaler
import scala.collection.mutable.{MutableList => ML}

In [ ]:
val deltaOperationARXMultiOutput = (deltaT: List[Int], deltaTargets: List[Int]) =>
    DataPipe((lines: Iterable[(Double, DenseVector[Double])]) =>
      lines.toList.sliding(math.max(deltaT.max, deltaTargets.max) + 1).map((history) => {

        val hist = history.take(history.length - 1).map(_._2)

        val num_outputs = deltaTargets.length

        val featuresAcc: ML[Double] = ML()

        val lags = deltaT ++ deltaTargets

        (0 until hist.head.length).foreach((dimension) => {
          //for each dimension/regressor take points t to t-order
          featuresAcc ++= hist.takeRight(lags(dimension))
            .map(vec => vec(dimension))
        })

        val outputs = history.last._2(0 until num_outputs)
        val features = DenseVector(featuresAcc.toArray)

        (features, outputs)
      }).toStream)

In [ ]:
val deltaT = 2

In [ ]:
val preProcessPipe = fileToStream >
  trimLines >
  replaceWhiteSpaces >
  extractTrainingFeatures(
    List(0,5,6,7,8,1,2,3,4),
    Map()
  ) >
  removeMissingLines >
  IterableDataPipe((line: String) => {
    val splits = line.split(",")
    val timestamp = splits.head.toDouble
    val data_vec = DenseVector(splits.tail.map(_.toDouble))
    (timestamp, data_vec)
  }) >
  deltaOperationARXMultiOutput(
    List.fill(4)(deltaT),
    List.fill(4)(deltaT))

In [ ]:
preProcessPipe("../data/steamgen.csv")